In [5]:
import matplotlib.pyplot as plt

from pyrocko import util, model, io, trace, moment_tensor, gmtpy
from pyrocko import pz
from pyrocko import orthodrome as od
from pyrocko.io import quakeml
from pyrocko.io import stationxml as fdsn
from pyrocko.client import catalog
from pyrocko.automap import Map

from obspy.clients.fdsn.client import Client
from obspy import UTCDateTime
from obspy.core.event import Catalog
from obspy.core.stream import Stream
from obspy.core.event import Event
from obspy.core.event import Origin
from obspy.core.event import Magnitude
from obspy import read
from obspy import read_events
from obspy import read_inventory
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os
import pickle

workdir='../'
catdir =  os.path.join(workdir,'CAT')
meta_datadir=os.path.join(workdir,'META_DATA')
datadir=os.path.join(workdir,'DATA')

In [11]:
#turkey eq parameters
eq_name='turkey_'
stime=UTCDateTime('2023-02-01T00:00:00')
etime=UTCDateTime('2023-02-10T00:00:00')

min_lat=30.0
max_lat=45.0
min_long=25.0
max_long=45.0

In [2]:
#japan eq parameters
stime=UTCDateTime('2023-12-30T00:00:00')
etime=UTCDateTime('2024-01-03T00:00:00')

min_lat=30.0
max_lat=41.0
min_long=127.0
max_long=144.0

In [3]:
#DOWNLOAD CATALOGUE
client=Client('IRIS')

cat_eq=client.get_events(starttime=stime,endtime=etime,
                minlatitude=min_lat, maxlatitude=max_lat,
                minlongitude=min_long, maxlongitude=max_long,
                minmagnitude=7.0)

print(cat_eq)

2 Event(s) in Catalog:
2023-02-06T10:24:49.258000Z | +38.023,  +37.203 | 7.5  mww
2023-02-06T01:17:34.362000Z | +37.225,  +37.021 | 7.8  mww


In [13]:
# reformat local catalog from INGV
catname = os.path.join(catdir, 'cat_eq_turkey.pf')

events = []

for ev in cat_eq:
    timetmp= str(ev.origins[0].time)
    timestr = timetmp[0:10] + ' ' + timetmp[11:23]                                        # format: '2010-01-01 00:00:00.000'
    time = util.str_to_time(timestr)
    event_name = eq_name + timestr[0:4] +'_'+ timestr[5:7]+'_'+timestr[8:10]+'_'+timestr[11:13]+'_'+timestr[14:16]+'_'+timestr[17:19]

    lat=    float( ev.origins[0].latitude )
    lon =   float( ev.origins[0].longitude )
    depth = float( ev.origins[0].depth )
    magnitude = float( ev.magnitudes[0].mag )
    events.append(model.Event(name=event_name, time=time,
                              lat=lat, lon=lon,
                              depth=depth, magnitude=magnitude))
events.sort(key=lambda x: x.time, reverse=False)
model.dump_events(events, catname)
print('Number of events:', len(events))

Number of events: 2


In [14]:
#read catalogue
catname = os.path.join(catdir, 'cat_eq_turkey.pf')

cat = model.load_events(catname)
print('Number of events:',len(cat))

number of events: 2


In [6]:
client=Client('INGV')
stations_name=os.path.join(meta_datadir, 'stations_flegrei_INGV.xml')
stations=read_inventory(stations_name)                                 #read

print(stations)

Inventory created at 2024-01-16T14:42:48.667000Z
	Created by: INGV-ONT WEB SERVICE: fdsnws-station | version: 1.1.57
		    /exist/apps/fdsn-station/fdsnws/station/1/query?starttime=2014-01-...
	Sending institution: eXistDB (INGV-ONT)
	Contains:
		Networks (3):
			IV, IX, ZM
		Stations (49):
			IV.CAAM (Accademia Aeronautica Pozzuoli)
			IV.CAFL (Arco Felice)
			IV.CAWE (Astroni Ovest - Stazione Osservatorio Vesuviano)
			IV.CBAC (Campi Flegrei, Castello di Baia - Stazione Osservatorio Vesu)
			IV.CCAP (Capo Posillipo C. acqisiz. - Stazione Osservatorio Vesuviano)
			IV.CFMN (Monte Nuovo)
			IV.CMIS (Faro di Capo Miseno - Stazione Osservatorio Vesuviano)
			IV.CMSN (Monte S.Angelo)
			IV.CMTS (Campi Flegrei, Via Metastasio CNR-IRC)
			IV.CNIS (Campi Flegrei - Isola di Nisida)
			IV.COLB (Campi Flegrei, Galleria Monte Olibano, Stazione Osservatorio)
			IV.CPIS (Campi Flegrei - Fumarola Pisciarelli)
			IV.CPOZ (Darsena Pozzuoli - Stazione Osservatorio Vesuviano)
			IV.CQUE (Campi Flegrei 

In [ ]:
#WAVEFORMS
client=Client('INGV')

for ev in cat_eq:
    
    origin_time=ev.origins[0].time
    evID=ev.resource_id.id.split('=')[1]
    print('event id: ',evID)
    print('origin time event:',origin_time)
    print('extimated magnitude:',ev.magnitudes[0].mag)

    event_start=origin_time +0
    event_end=origin_time +7200

    wave=Stream()
    for network in inv_f:
        for  station in network.stations:
            try:
                wave += client.get_waveforms(starttime=event_start,endtime=event_end,
                                    network=network.code,station=station.code,location='*', channel='HH?',
                                    attach_response=True,)
            except:
                print(station.code , 'station not recording')
                continue

    print('traces found:',len(wave.traces))

    '''
    wave.merge(fill_value=0)
    # trim over the [t1, t2] interval
    wave.trim(starttime=event_start, endtime=event_end, pad=True, fill_value=0)

    
    # remove trend
    wave.detrend("demean")
    
    #remove instrumental response
    pre_filt = [0.01, 0.1, 25,30]
    
    #remove instrumental response
    wave.remove_response(inventory=inv_f, output='DISP', pre_filt=pre_filt) #output=VEL ??
    '''
    
    
    try:
        wave.write('waveform/big_eq_polarity/wave_'+ evID +'.mseed',format='MSEED')
        print('saved!')
    except:
        continue
    

# Process and remove instrumental response (optional)

In [5]:
#WAVEFORMS
client=Client('INGV')

for ev in cat_eq:
    
    origin_time=ev.origins[0].time
    evID=ev.resource_id.id.split('=')[1]
    print('event id: ',evID)
    print('origin time event:',origin_time)
    print('extimated magnitude:',ev.magnitudes[0].mag)

    event_start=origin_time +0
    event_end=origin_time +7200

    wave=Stream()
    for network in inv_f:
        for  station in network.stations:
            try:
                wave += client.get_waveforms(starttime=event_start,endtime=event_end,
                                    network=network.code,station=station.code,location='*', channel='HH?',
                                    attach_response=True,)
            except:
                print(station.code , 'station not recording')
                continue

    print('traces found:',len(wave.traces))

    
    wave.merge(fill_value=0)
    # trim over the [t1, t2] interval
    wave.trim(starttime=event_start, endtime=event_end, pad=True, fill_value=0)

    
    # remove trend
    wave.detrend("demean")
    
    #remove instrumental response
    pre_filt = [0.01, 0.1, 25,30]
    
    #remove instrumental response
    wave.remove_response(inventory=inv_f, output='DISP', pre_filt=pre_filt) # if I want output=VEL ??
    
    
    
    try:
        wave.write('waveform/big_eq_polarity/wave_no_instrument_'+ evID +'.mseed',format='MSEED')
        print('saved!')
    except:
        continue
    

event id:  11786236
origin time event: 2024-01-01T07:10:09.692000Z
extimated magnitude: 7.5
CAFL station not recording
CAWE station not recording
CCAP station not recording
CMTS station not recording
IBCM station not recording
ICVJ station not recording
IMTC station not recording
IPSM station not recording
IVLC station not recording
MODR station not recording
VARP station not recording
VTIR station not recording
VTRZ station not recording
NAPI station not recording
T1361 station not recording
T1363 station not recording
T1365 station not recording
T1366 station not recording
T1367 station not recording
T1368 station not recording
traces found: 597


ValueError: No matching response information found.